# Pregunta 3

This exercise is based on the paper, Eissa, Nada, and Jeffrey B. Liebman. 1996. **Labor Supply
Responses to the Earned Income Tax Credit (EITC)**. Quarterly Journal of Economics. 111(2):
605-637. <br>The data “eitc.dta” file contains data for single women 20-54 with less than high school
education, as this group is most likely to be affected by the EITC.

#### Cargar datos

In [108]:
import pandas as pd
import pyreadstat
import statsmodels.formula.api as smf

In [111]:
eitc = pd.read_stata("https://github.com/MendozaC25/09-Econometria_Aplic._Av/raw/refs/heads/main/P.Calificada_1/eitc.dta")
eitc.head()

,state,year,urate,children,nonwhite,finc,earn,age,ed,work,unearn
0,11.0,1991.0,7.6,0,1,18714.394273,18714.394273,26,10,1,0.000000
1,12.0,1991.0,7.2,1,0,4838.568282,471.365639,22,9,1,4.367203
2,13.0,1991.0,6.4,2,0,8178.193833,0.000000,33,11,0,8.178194
3,14.0,1991.0,9.1,0,1,9369.570485,0.000000,43,11,0,9.369570
4,15.0,1991.0,8.6,3,1,14706.607930,14706.607930,23,7,1,0.000000


#### 1. [3 points ] Create a table summarizing all the data provided in the data set.

In [113]:
eitc.describe()

,state,year,urate,children,nonwhite,finc,earn,age,ed,work,unearn
count,13746.00000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000
mean,54.52459,1993.347046,6.761734,1.192638,0.600684,15255.319310,10432.475514,35.209661,8.806053,0.513022,4.822844
std,27.13521,1.709800,1.462461,1.382105,0.489776,19444.249684,18200.758138,10.157128,2.635639,0.499849,7.122624
min,11.00000,1991.000000,2.600000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000
25%,31.00000,1992.000000,5.700000,0.000000,0.000000,5123.417782,0.000000,26.000000,7.000000,0.000000,0.000000
50%,56.00000,1993.000000,6.800000,1.000000,1.000000,9636.664360,3332.179931,34.000000,10.000000,1.000000,2.973197
75%,81.00000,1995.000000,7.700000,2.000000,1.000000,18659.178178,14321.223709,44.000000,11.000000,1.000000,6.863863
max,95.00000,1996.000000,11.400000,9.000000,1.000000,575616.820870,537880.611850,54.000000,11.000000,1.000000,134.057520


#### 2. [5 points ] Calculate the sample means of all variables for <br>(a) single women with no children, <br>(b) single women with 1 child, and <br>(c) single women with 2+ children.


In [124]:
no_children = eitc.loc[eitc['children'] == 0]
one_children = eitc.loc[eitc['children'] == 1]
two_children = eitc.loc[eitc['children'] >= 2]

(a) single women with no children

In [138]:
m_none = no_children.iloc[:,[2,3,5,6,7,8,9,10]].mean()
m_none = m_none.reset_index()
m_none.columns = ['Variable', 'Promedio']
m_none

,Variable,Promedio
0,urate,6.663067
1,children,0.000000
2,finc,18559.863110
3,earn,13760.255844
4,age,38.498228
5,ed,8.548676
6,work,0.574490
7,unearn,4.799607


(b) single women with 1 child

In [128]:
m_one = one_children.iloc[:,[2,3,5,6,7,8,9,10]].mean()
m_one = m_one.reset_index()
m_one.columns = ['Variable', 'Promedio']
m_one

,Variable,Promedio
0,urate,6.802061
1,children,1.000000
2,finc,13941.569547
3,earn,9928.278759
4,age,33.758993
5,ed,8.992479
6,work,0.537606
7,unearn,4.013291


(c) single women with 2+ children

In [130]:
m_two = two_children.iloc[:,[2,3,5,6,7,8,9,10]].mean()
m_two = m_two.reset_index()
m_two.columns = ['Variable', 'Promedio']
m_two

,Variable,Promedio
0,urate,6.858665
1,children,2.801092
2,finc,11985.295298
3,earn,6613.546648
4,age,32.047469
5,ed,9.006721
6,work,0.420710
7,unearn,5.371749


#### 3. [5 points ] Construct a variable for the “treatment” called **anykids** (indicator for 1 or more kids) <br>and a variable for time being after the expansion (called **post93**, should be 1 for 1994 and later).


##### Variable "anykids" 

In [140]:
#Se asignará el valor de 1 para indicar que tiene hijos, y 0 en caso no los tenga

def treatment(var):
    if var >= 1:
        return 1
    else:
        return 0

In [142]:
eitc['anykids'] = eitc['children'].apply(treatment)

In [146]:
# Comprobamos que se ha creado correctamente la variable

eitc.iloc[:,[3,11]].head()

,children,anykids
0,0,0
1,1,1
2,2,1
3,0,0
4,3,1


##### Variable "post93"

In [157]:
#Se asignará el valor de 1 para indicar que es posterior a 1993, y 0 en caso no lo sea

def post(var):
    if var > 1993:
        return 1
    else:
        return 0

In [159]:
eitc['post93'] = eitc['year'].apply(post)

In [173]:
# Comprobamos que se ha creado correctamente la variable

eitc.iloc[151:,[1,12]].head()

,year,post93
151,1993.0,0
152,1993.0,0
153,1994.0,1
154,1994.0,1
155,1994.0,1


#### 4. [10 points ] Using the “interaction term” diff-in-diff specification, run a regression to <br>estimate the difference-in-differences estimate of the effect of the EITC program on earnings. <br>Use all women with children as the treatment group.


In [175]:
# Creamos el término de interacción

eitc['int_term'] = eitc['anykids'] * eitc['post93']

In [177]:
# Corremos la regresión Diff in Diff

modelo1 = smf.ols('earn ~ anykids + post93 + int_term', data=eitc).fit()
print(modelo1.summary())

                            OLS Regression Results                            
Dep. Variable:                   earn   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     124.3
Date:                Wed, 27 Nov 2024   Prob (F-statistic):           1.96e-79
Time:                        12:28:22   Log-Likelihood:            -1.5416e+05
No. Observations:               13746   AIC:                         3.083e+05
Df Residuals:                   13742   BIC:                         3.084e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    1.38e+04    319.810     43.146      0.0

#### 5. [7 points ] Repeat (iv), but now include ***state*** and **year** fixed effects <br>[Hint: state fixed effects, are included when we include a dummy variable for each state]. <br>Do you get similar estimated treatment effects compared to (iv)?


##### Dummies para **state** y **year**

#### 6. [7 points] Using the specification from (v), re-estimate this model including <br>urate nonwhite age ed unearn, as well as state and year FEs as controls. <br>Do you get similar estimated treatment effects compared to (v)?


#### 7. [7 points ] Estimate a version of your model that allows the treatment effect to vary by those with 1 or 2+ children. <br>Include all other variables as in (vi). <br>Does the intervention seem to be more effective for one of these groups over the other? <br>Why might this be the case in the real world?


#### 8. [6 points ] Estimate a “placebo” treatment model as follows: Take data from only the prereform <br> period (up to and including 1993). <br>Drop the rest, or restrict your model to run only if **year** <= 1993. <br>Estimate the effect for all affected women together, just as in (vi). <br>Introduce a placebo policy that begins in 1992 (so 1992 and 1993 are both “treated” with this fake policy). <br>What do you find? <br>Are your results “reassuring”?